https://stepik.org/lesson/303592/step/7

In [ ]:
import torch
import torch.nn as nn


eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    # Напишите в этом месте нормирование тензора.
    # требуется нормировать данные не по одной оси, а по всем размерностям, кроме нулевой
    # Получаем размерность тензора
    input_tensor_shape = input_tensor.shape

    # # Способ 1. Не работает на платформе!
    # # Определяем оси для нормализации (все, кроме нулевой)
    # axes_to_normalize = tuple(range(1, len(input_tensor_shape)))
    # # Нормализуем тензор
    # mean = input_tensor.mean(dim=axes_to_normalize, keepdim=True)
    # var = input_tensor.var(dim=axes_to_normalize, keepdim=True, unbiased=False)
    # normalized_tensor = (input_tensor - mean) / torch.sqrt(var + eps)
    # return normalized_tensor

    # Способ 2. Через .reshape
    # Получаем размерность тензора
    input_tensor_shape = input_tensor.shape

    #  Создаем новый тензор, переформированный в (N, -1),
    #  где N - количество образцов, а -1 означает, что оставшиеся размерности
    #  будут вычислены автоматически.
    reshaped_tensor = input_tensor.reshape(input_tensor_shape[0], -1)

    # Вычисляем среднее значение и стандартное отклонение по всем осям, кроме нулевой
    mean = reshaped_tensor.mean(dim=1, keepdim=True)
    var = reshaped_tensor.var(dim=1, keepdim=True, unbiased=False)

    # Нормализуем данные
    normalized_tensor = (reshaped_tensor - mean) / torch.sqrt(var + eps)

    # Возвращаем тензор в исходную форму
    normalized_tensor = normalized_tensor.reshape(input_tensor_shape)
    return normalized_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    print(input_tensor.shape)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

torch.Size([3, 4])
torch.Size([3, 4, 5])
torch.Size([3, 4, 5, 6])
torch.Size([3, 4, 5, 6, 7])
torch.Size([3, 4, 5, 6, 7, 8])
torch.Size([3, 4, 5, 6, 7, 8, 9])
True
